# Multi-asset forecasting with an LSTM

This notebook contains code for training and forecasting 22 stocks from the OMXS30 index
with an LSTM model.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

from data.MultiAssetDataset import MultiAssetDataset
from multi_asset_models.MultiAssetLSTM import MultiAssetLSTM